In [1]:
!nvidia-smi

Fri Apr 11 16:11:57 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.133.07             Driver Version: 570.133.07     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 5080        Off |   00000000:65:00.0  On |                  N/A |
|  0%   47C    P8             25W /  360W |    1028MiB /  16303MiB |      4%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
#%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128

# Pre-release versions of PyTorch, TorchVision, and Torchaudio for CUDA 12.8
%pip install --pre torch==2.8.0.dev20250325+cu128 torchvision==0.22.0.dev20250325+cu128 torchaudio==2.6.0.dev20250325+cu128 --index-url https://download.pytorch.org/whl/nightly/cu128



Looking in indexes: https://download.pytorch.org/whl/cu128
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch

print(f"PyTorch version: {torch.__version__}")

# Check PyTorch has access to MPS (Metal Performance Shader, Apple's GPU architecture)
print(f"Is MPS (Metal Performance Shader) built? {torch.backends.mps.is_built()}")
print(f"Is MPS available? {torch.backends.mps.is_available()}")
print(f"Is CUDA available? {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
print(f"CUDA device count: {torch.cuda.device_count()}")

print(f"Current device: {torch.cuda.current_device()}")

# Set the device
device = (
    "mps"
    if torch.backends.mps.is_available()
    else "cuda" if torch.cuda.is_available() else None
)
if device is None:
    print("MPS or Cura is not available. Using CPU instead.")
    device = "cpu"

print(f"Using device: {device}")

PyTorch version: 2.8.0.dev20250325+cu128
Is MPS (Metal Performance Shader) built? False
Is MPS available? False
Is CUDA available? True
CUDA version: 12.8
CUDA device count: 1
Current device: 0
Using device: cuda


In [3]:
import os

HOME = os.getcwd()
print(HOME)

/home/daniel/Projects/Crudus/Heima/shop-price-tag


In [ ]:
%pip install -q roboflow
%pip install -q ultralytics
%pip install -q python-dotenv
%pip install -q supervision
%pip install -q onnxruntime

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
%load_ext dotenv
%dotenv

In [4]:
from dotenv import dotenv_values

config = dotenv_values(".env")

In [5]:
from roboflow import Roboflow

rf = Roboflow(api_key=config["ROBOFLOW_API_KEY"])
project = rf.workspace("daniel-kaldheim-4f58q").project("price-tags-c79pp")
dataset = project.version(4).download("yolov11")

loading Roboflow workspace...
loading Roboflow project...


In [4]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11m.pt")

# Train the model

results = model.train(
    data=HOME + "/Price-tags-4/data.yaml",
    project=HOME + "/Price-tags-4",
    name="yolo11m-price-tag",
    exist_ok=True,
    epochs=300,
    imgsz=640,
    device=device,
    save=True,
    save_period=10,
    val=True,
    patience=50,
    workers=8,
    batch=-1,
    amp=True,
)

Ultralytics 8.3.107 🚀 Python-3.11.12 torch-2.8.0.dev20250325+cu128 CUDA:0 (NVIDIA GeForce RTX 5080, 15848MiB)
engine/trainer: task=detect, mode=train, model=yolo11m.pt, data=/home/daniel/Projects/Crudus/Heima/shop-price-tag/Price-tags-4/data.yaml, epochs=300, time=None, patience=50, batch=-1, imgsz=640, save=True, save_period=10, cache=False, device=cuda, workers=8, project=/home/daniel/Projects/Crudus/Heima/shop-price-tag/Price-tags-4, name=yolo11m-price-tag, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, s

train: Scanning /home/daniel/Projects/Crudus/Heima/shop-price-tag/Price-tags-4/train/labels.cache... 930 images, 0 backgrounds, 0 corrupt: 100%|██████████| 930/930 [00:00<?, ?it/s]

AutoBatch: Computing optimal batch size for imgsz=640 at 60.0% CUDA memory utilization.
AutoBatch: CUDA:0 (NVIDIA GeForce RTX 5080) 15.48G total, 0.21G reserved, 0.19G allocated, 15.07G free
      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output


    20054550       68.19         1.634         54.79         256.5        (1, 3, 640, 640)                    list
    20054550       136.4         2.244         30.92         139.9        (2, 3, 640, 640)                    list
    20054550       272.8         3.379         34.27         152.9        (4, 3, 640, 640)                    list
    20054550       545.5         5.595         35.03         154.4        (8, 3, 640, 640)                    list
    20054550        1091         9.896         46.81           164       (16, 3, 640, 640)                    list
AutoBatch: Using batch-size 16 for CUDA:0 9.52G/15.48G (61%) ✅


train: Scanning /home/daniel/Projects/Crudus/Heima/shop-price-tag/Price-tags-4/train/labels.cache... 930 images, 0 backgrounds, 0 corrupt: 100%|██████████| 930/930 [00:00<?, ?it/s]
val: Scanning /home/daniel/Projects/Crudus/Heima/shop-price-tag/Price-tags-4/valid/labels.cache... 87 images, 0 backgrounds, 0 corrupt: 100%|██████████| 87/87 [00:00<?, ?it/s]


...

    214/300      10.1G     0.4551     0.2683     0.8262         23        640: 100%|██████████| 59/59 [00:08<00:00,  6.87it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  6.97it/s]

                   all         87        420       0.93      0.983      0.983      0.829
EarlyStopping: Training stopped early as no improvement observed in last 50 epochs. Best results observed at epoch 164, best model saved as best.pt.
To update EarlyStopping(patience=50) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



214 epochs completed in 0.587 hours.
Optimizer stripped from /home/daniel/Projects/Crudus/Heima/shop-price-tag/Price-tags-4/yolo11m-price-tag/weights/last.pt, 40.5MB
Optimizer stripped from /home/daniel/Projects/Crudus/Heima/shop-price-tag/Price-tags-4/yolo11m-price-tag/weights/best.pt, 40.5MB

Validating /home/daniel/Projects/Crudus/Heima/shop-price-tag/Price-tags-4/yolo11m-price-tag/weights/best.pt...
Ultralytics 8.3.107 🚀 Python-3.11.12 torch-2.8.0.dev20250325+cu128 CUDA:0 (NVIDIA GeForce RTX 5080, 15848MiB)
YOLO11m summary (fused): 125 layers, 20,031,574 parameters, 0 gradients, 67.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  2.88it/s]


                   all         87        420      0.933      0.989      0.986      0.837
       price-tag-price         87        216      0.943      0.987      0.983      0.738
            price-tags         87        204      0.924       0.99      0.989      0.935
Speed: 0.1ms preprocess, 2.4ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to /home/daniel/Projects/Crudus/Heima/shop-price-tag/Price-tags-4/yolo11m-price-tag


In [ ]:
from PIL import Image
from ultralytics import YOLO
import matplotlib.pyplot as plt
import cv2
import supervision as sv

# Load a model
# model = YOLO('yolov8n.pt')  # load an official model
model = YOLO("Price-tags-4/yolo11m-price-tag/weights/best.pt")  # load a custom model

IMAGE_NAME = "test/rema/IMG_0734.jpeg"
IMAGE_PATH = os.path.join(HOME, IMAGE_NAME)
cv_image = cv2.imread(IMAGE_PATH)

# Predict with the model
# results = model('Price-tags-2/valid/images/IMG_0734_jpeg.rf.5f7c304d33522c57f8d8a06c9ba8a062.jpg')  # predict on an image
results = model(cv_image)  # predict on an image

for r in results:
    im_array = r.plot()  # plot a BGR numpy array of predictions
    im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
    im.save("results.jpg")  # save image
    sv.plot_image(im_array, (16, 16))


0: 480x640 6 price-tag-prices, 6 price-tagss, 42.9ms
Speed: 3.0ms preprocess, 42.9ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)


<Figure size 1600x1600 with 1 Axes>

In [7]:
from PIL import Image
from ultralytics import YOLO
import matplotlib.pyplot as plt
import cv2
import supervision as sv
import os

# Load the model
model = YOLO("Price-tags-4/yolo11m-price-tag/weights/best.pt")  # load a custom model

IMAGE_NAME = "test/rema/IMG_0734.jpeg"
IMAGE_PATH = os.path.join(HOME, IMAGE_NAME)
cv_image = cv2.imread(IMAGE_PATH)

sv.plot_image(cv_image)

# Predict with the model
results = model(cv_image)  # predict on an image

for r in results:
    for dim in r.boxes.xyxy:
        x, y, x1, y1 = dim.tolist()
        image = cv_image[round(y) : round(y1), round(x) : round(x1)]
        sv.plot_image(image, (16, 16))

<Figure size 1200x1200 with 1 Axes>


0: 640x480 2 price-tag-prices, 2 price-tagss, 10.7ms
Speed: 2.3ms preprocess, 10.7ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 480)


<Figure size 1600x1600 with 1 Axes>

<Figure size 1600x1600 with 1 Axes>

<Figure size 1600x1600 with 1 Axes>

<Figure size 1600x1600 with 1 Axes>

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("Price-tags-4/yolo11m-price-tag/weights/best.pt")  # load a custom model

model.export(format="onnx")

Ultralytics 8.3.40  Python-3.11.1 torch-2.1.0+cu121 CPU (12th Gen Intel Core(TM) i9-12900KF)
YOLO11n summary (fused): 238 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs

PyTorch: starting from 'runs\detect\train\weights\best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 6, 8400) (5.3 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0', 'onnxslim', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
   ---------------------------------------- 14.5/14.5 MB 41.6 MB/s eta 0:00:00
   --------------------------------------- 279.7/279.7 MB 26.1 MB/s eta 0:00:00

requirements: AutoUpdate success  27.4s, installed 3 packages: ['onnx>=1.12.0', 'onnxslim', 'onnxruntime-gpu']
requirements:  Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.17.0 opset 17...
ONNX: slimming with onnxslim 0.1.43...
ONNX: export success  28.5s, saved as 'runs\detect\train\weights\best.onnx' (10.1 MB)

Export complete (30.1s)
Results sav

'runs\\detect\\train\\weights\\best.onnx'

In [22]:
#!yolo mode=export model="runs/detect/train/weights/best.pt" format=torchscript optimize
!yolo export model=runs/detect/train/weights/best.pt format=onnx

Ultralytics 8.3.40 🚀 Python-3.11.1 torch-2.1.0+cu121 CPU (12th Gen Intel Core(TM) i9-12900KF)
YOLO11n summary (fused): 238 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs

PyTorch: starting from 'runs\detect\train\weights\best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 6, 8400) (5.3 MB)

ONNX: starting export with onnx 1.17.0 opset 17...
ONNX: slimming with onnxslim 0.1.43...
ONNX: export success ✅ 1.0s, saved as 'runs\detect\train\weights\best.onnx' (10.1 MB)

Export complete (1.4s)
Results saved to C:\Users\danie\Projects\shop-price-tag\runs\detect\train\weights
Predict:         yolo predict task=detect model=runs\detect\train\weights\best.onnx imgsz=640  
Validate:        yolo val task=detect model=runs\detect\train\weights\best.onnx imgsz=640 data=c:\Users\danie\Projects\shop-price-tag/Price-tags-4/data.yaml  
Visualize:       https://netron.app
💡 Learn more at https://docs.ultralytics.com/modes/export


In [3]:
!yolo export format=tflite model=runs/detect/train/weights/best.pt imgsz=320 int8

Ultralytics 8.3.40 🚀 Python-3.11.1 torch-2.1.0+cu121 CPU (12th Gen Intel Core(TM) i9-12900KF)
WARNING ⚠️ INT8 export requires a missing 'data' arg for calibration. Using default 'data=coco8.yaml'.
YOLO11n summary (fused): 238 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs

PyTorch: starting from 'runs\detect\train\weights\best.pt' with input shape (1, 3, 320, 320) BCHW and output shape(s) (1, 6, 2100) (5.3 MB)
requirements: Ultralytics requirements ['tf_keras', 'sng4onnx>=1.0.1', 'onnx_graphsurgeon>=0.3.26', 'onnx2tf>1.17.5,<=1.22.3', 'tflite_support'] not found, attempting AutoUpdate...
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com











  Preparing metadata (setup.py): started

  Preparing metadata (setup.py): finished with status 'done'













































   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 1.7/1.7 MB 45.7 MB/s eta 0:00:00









  Created w



  0%|          | 0.00/1.11M [00:00<?, ?B/s]
100%|██████████| 1.11M/1.11M [00:00<00:00, 21.8MB/s]

Unzipping calibration_image_sample_data_20x128x128x3_float32.npy.zip to C:\Users\danie\Projects\shop-price-tag\calibration_image_sample_data_20x128x128x3_float32.npy...:   0%|          | 0/1 [00:00<?, ?file/s]
Unzipping calibration_image_sample_data_20x128x128x3_float32.npy.zip to C:\Users\danie\Projects\shop-price-tag\calibration_image_sample_data_20x128x128x3_float32.npy...: 100%|██████████| 1/1 [00:00<00:00, 95.23file/s]

  0%|          | 0.00/433k [00:00<?, ?B/s]
100%|██████████| 433k/433k [00:00<00:00, 12.6MB/s]

Unzipping C:\Users\danie\Projects\shop-price-tag\datasets\coco8.zip to C:\Users\danie\Projects\shop-price-tag\datasets\coco8...:   0%|          | 0/25 [00:00<?, ?file/s]
Unzipping C:\Users\danie\Projects\shop-price-tag\datasets\coco8.zip to C:\Users\danie\Projects\shop-price-tag\datasets\coco8...: 100%|██████████| 25/25 [00:00<00:00, 2628.67file/s]

Scanning C:\Users\danie\

In [4]:
!yolo export format=mlmodel model=runs/detect/train/weights/best.pt imgsz=[320, 192] half nms

Ultralytics 8.3.40 🚀 Python-3.11.1 torch-2.1.0+cu121 CPU (12th Gen Intel Core(TM) i9-12900KF)
YOLO11n summary (fused): 238 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs

PyTorch: starting from 'runs\detect\train\weights\best.pt' with input shape (1, 3, 320, 192) BCHW and output shape(s) (1, 6, 1260) (5.3 MB)
requirements: Ultralytics requirement ['coremltools>=6.0,<=6.2'] not found, attempting AutoUpdate...


     ---------------------------------------- 0.0/909.5 kB ? eta -:--:--
     ------------------------------------- 909.5/909.5 kB 10.3 MB/s eta 0:00:00

  Preparing metadata (setup.py): started

  Preparing metadata (setup.py): finished with status 'done'













  Created wheel for coremltools: filename=coremltools-6.2-py3-none-any.whl size=1212821 sha256=9a4c04be1dad01bc0c6a3e659e3c1d54fe43daa04fe2a3e11cd1450b8127e884

  Stored in directory: C:\Users\danie\AppData\Local\Temp\pip-ephem-wheel-cache-5630jcgj\wheels\2c\13\ee\9de6b8bff9d8163f9df1f41cec2dd2a724122e4a57da

TensorFlow version 2.18.0 has not been tested with coremltools. You may run into unexpected errors. TensorFlow 2.10.0 is the most recent version that has been tested.
Torch version 2.1.0+cu121 has not been tested with coremltools. You may run into unexpected errors. Torch 1.13.1 is the most recent version that has been tested.
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\danie\AppData\Local\Programs\Python\Python311\Scripts\yolo.exe\__main__.py", line 7, in <module>
  File "C:\Users\danie\AppData\Local\Programs\Python\Python311\Lib\site-packages\ultralytics\cfg\__init__.py", line 972, in entrypoint
    getattr(model, mode)(**overrides)  # default args from model
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\danie\AppData\Local\Programs\Python\Python311\Lib\site-packages\ultralytics\engine\model.py", line 737, in export
    return Exporter(overrides=args, _callbacks=s